In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from dtaidistance import dtw
import itertools
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

import os

import random

from SVM_models.models import evaluate_binary_SVM_standard, evaluate_binary_SVM_custom

# import GunPoint dataset from GunPoint/GunPoint_test.txt and GunPoint/GunPoint_train.txt


def load_dataset(file_path):
    with open(file_path) as f:
        lines = f.readlines()
    # Parse each line, separating the label and features
    labels = []
    features = []
    for line in lines:
        values = list(map(float, line.strip().split()))
        labels.append(int(values[0])-1)  # First value is the label, cast to int
        features.append(values[1:])    # Remaining values are features
    # Convert lists to numpy arrays for easier handling
    labels = np.array(labels)
    features = np.array(features)
    # normalize dataset to 0 1
    features = (features - features.min()) / (features.max() - features.min())
    return labels, features

# Load the GunPoint dataset

#dataset_path with join

train_path = os.path.join("GunPoint", "GunPoint_TRAIN.txt")
test_path = os.path.join("GunPoint", "GunPoint_TEST.txt")

train_labels, train_features = load_dataset(train_path)
INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES = load_dataset(test_path)

# split test in validation (50) and test (100)



In [2]:
EXPS = [
    "RANDOM",
    "RANDOM_BEST",
    "BAYESIAN_HAMILTONIAN",
    "BAYESIAN_MODEL",
]

OPT_TIMES = 30

CONFIG = {
    "N_QUBITS": 2,
    "SPARSITY": 0.95,
    "ENCODING": "euler",
}

SEEDS = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
EXP = "BAYESIAN_MODEL_TKA"


In [3]:
import os
def save_results(EXP, encoding, seed, train_accuracy, val_accuracy, test_accuracy, Hamiltonian_c, hidden_alphas):
    # Create folder structure
    folder_name = f"{EXP}__{encoding}"
    seed_folder = os.path.join("exps_results", folder_name, f"exp_{seed}")
    os.makedirs(seed_folder, exist_ok=True)

    # Save configuration and OPT_TIMES to a text file
    with open(os.path.join(seed_folder, "config.txt"), "w") as f:
        f.write(f"Experiment: {EXP}\n")
        f.write(f"Encoding: {encoding}\n")
        f.write(f"Number of Qubits: {CONFIG['N_QUBITS']}\n")
        f.write(f"Sparsity Coefficient: {CONFIG['SPARSITY']}\n")
        f.write(f"OPT_TIMES: {OPT_TIMES}\n")

    # Save train, validation, and test accuracy
    with open(os.path.join(seed_folder, "train_accuracy.txt"), "w") as f:
        f.write(f"Train Accuracy: {train_accuracy}\n")
    
    with open(os.path.join(seed_folder, "val_accuracy.txt"), "w") as f:
        f.write(f"Validation Accuracy: {val_accuracy}\n")

    with open(os.path.join(seed_folder, "test_accuracy.txt"), "w") as f:
        f.write(f"Test Accuracy: {test_accuracy}\n")

    # Save Hamiltonian_c and hidden_alphas
    with open(os.path.join(seed_folder, "Hamiltonian_c.txt"), "w") as f:
        f.write(str(Hamiltonian_c))

    with open(os.path.join(seed_folder, "hidden_alphas.txt"), "w") as f:
        f.write(str(hidden_alphas))


In [4]:
from SVM_models.QuantumSVM import QuantumSVM
from sklearn.metrics import accuracy_score

In [5]:
from bayes_opt import BayesianOptimization

I = np.array([[1, 0], [0, 1]])  # Identity
X = np.array([[0, 1], [1, 0]])  # Pauli-X
Y = np.array([[0, -1j], [1j, 0]])  # Pauli-Y
Z = np.array([[1, 0], [0, -1]]) 

L = 150

def generate_H_from_C(coefficients):
    """Generates a random Hamiltonian for the given number of qubits by combining Pauli matrices."""
    pauli_matrices = [I, X, Y, Z]  # Random coefficients for each term
    H = None
    # Generate all possible combinations of Pauli matrices for the given number of qubits
    for i in range(4**CONFIG["N_QUBITS"]):
        indices = np.unravel_index(i, (4,) * CONFIG["N_QUBITS"])
        kron_product = pauli_matrices[indices[0]]
        for idx in indices[1:]:
            kron_product = np.kron(kron_product, pauli_matrices[idx])
        kron_product = np.array(kron_product, dtype=complex)
        if H is None:
            H = coefficients[i] * kron_product
        else:
            H += coefficients[i] * kron_product
    H = H / np.linalg.norm(H)  # Normalize the Hamiltonian
    return H

def generate_random_alphas():
    return np.random.rand(L)

In [6]:
def target_kernel_alignment(predicted_kernel, target_kernel):
    # Normalize kernels
    predicted_norm = np.linalg.norm(predicted_kernel, 'fro')
    target_norm = np.linalg.norm(target_kernel, 'fro')
    
    # Compute alignment
    alignment = np.sum(predicted_kernel * target_kernel) / (predicted_norm * target_norm)
    return alignment

In [ ]:
for SEED in SEEDS:

    random.seed(SEED)

    test_labels, val_labels, test_features, val_features = train_test_split(INITIAL_TEST_LABELS, INITIAL_TEST_FEATURES, test_size=0.33, random_state=SEED)
    
    def objective_function(**kwargs):
        # Convert kwargs to a coefficient array
        coefficients = np.array([kwargs[f"coeff_{i}"] for i in range(4**CONFIG["N_QUBITS"])])
        alphas_c = np.array([kwargs[f"w_{i}"] for i in range(L)])
        #print("Coefficients:", coefficients)
        # Fit the QuantumSVM model with the generated Hamiltonian
        quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], Hamiltonian_c=coefficients, alphas_c=alphas_c, embedding_code=CONFIG["ENCODING"], sparsity_coefficient=CONFIG["SPARSITY"])
        quantum_svm.fit(train_features, train_labels)
        
        y_val = quantum_svm.predict(val_features)
        K_val = quantum_svm.K_test

        target_kernel = np.outer(val_labels, val_labels)
        

        alignment_score = target_kernel_alignment(K_val, target_kernel)
        return alignment_score


    pbounds = {f"coeff_{i}": (0, 1) for i in range(4**CONFIG["N_QUBITS"])}
    pbounds.update({f"w_{i}": (0, 1) for i in range(L)})

    # Initialize Bayesian Optimizer
    optimizer = BayesianOptimization(
        f=objective_function,
        pbounds=pbounds,
        random_state=SEED,
        verbose=2
    )

    optimizer.maximize(
        init_points=5,  # Number of random initial points
        n_iter=OPT_TIMES-5        # Number of optimization steps
    )
    # Retrieve the best coefficients
    best_coefficients = [optimizer.max["params"][f"coeff_{i}"] for i in range(4**CONFIG["N_QUBITS"])]
    best_alphas = [optimizer.max["params"][f"w_{i}"] for i in range(L)]

    #convert best_alphas to numpy array
    best_alphas = np.array(best_alphas)

    print("Best Coefficients:", best_coefficients)
    print("Best Val Accuracy:", optimizer.max["target"])

    quantum_svm = QuantumSVM(n_qubits=CONFIG["N_QUBITS"], Hamiltonian_c=best_coefficients, alphas_c=best_alphas, embedding_code=CONFIG["ENCODING"], sparsity_coefficient=CONFIG["SPARSITY"])

    quantum_svm.fit(train_features, train_labels)
    y_train = quantum_svm.predict_on_train()
    train_accuracy = accuracy_score(train_labels, y_train)
    print(f"Train Accuracy: {train_accuracy}")

    y_val = quantum_svm.predict(val_features)
    val_accuracy = accuracy_score(val_labels, y_val)
    print(f"Validation Accuracy: {val_accuracy}")

    y_test = quantum_svm.predict(test_features)
    accuracy = accuracy_score(test_labels, y_test)
    print(f"Test Accuracy: {accuracy}")


    # now I want to save the model Hamiltonian_c and hidden_alphas
    print("Saving model...")
    Hamiltonian_c = quantum_svm.Hamiltonian_c
    hidden_alphas = quantum_svm.hidden_alphas

    #print("Hamiltonian_c: ", Hamiltonian_c)
    #print("hidden_alphas: ", hidden_alphas)

    save_results(EXP, CONFIG["ENCODING"], SEED, train_accuracy, val_accuracy, accuracy, Hamiltonian_c, hidden_alphas)

|   iter    |  target   |  coeff_0  |  coeff_1  | coeff_10  | coeff_11  | coeff_12  | coeff_13  | coeff_14  | coeff_15  |  coeff_2  |  coeff_3  |  coeff_4  |  coeff_5  |  coeff_6  |  coeff_7  |  coeff_8  |  coeff_9  |    w_0    |    w_1    |   w_10    |   w_100   |   w_101   |   w_102   |   w_103   |   w_104   |   w_105   |   w_106   |   w_107   |   w_108   |   w_109   |   w_11    |   w_110   |   w_111   |   w_112   |   w_113   |   w_114   |   w_115   |   w_116   |   w_117   |   w_118   |   w_119   |   w_12    |   w_120   |   w_121   |   w_122   |   w_123   |   w_124   |   w_125   |   w_126   |   w_127   |   w_128   |   w_129   |   w_13    |   w_130   |   w_131   |   w_132   |   w_133   |   w_134   |   w_135   |   w_136   |   w_137   |   w_138   |   w_139   |   w_14    |   w_140   |   w_141   |   w_142   |   w_143   |   w_144   |   w_145   |   w_146   |   w_147   |   w_148   |   w_149   |   w_15    |   w_16    |   w_17    |   w_18    |   w_19    |    w_2    |   w_20    |   w_21    |   

Building TRAIN_KERNEL:   0%|          | 0/1225 [00:00<?, ?it/s]

Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:56<00:00, 21.46it/s]


| 1         | 0.4265    | 0.5488    | 0.7152    | 0.6028    | 0.5449    | 0.4237    | 0.6459    | 0.4376    | 0.8918    | 0.9637    | 0.3834    | 0.7917    | 0.5289    | 0.568     | 0.9256    | 0.07104   | 0.08713   | 0.02022   | 0.8326    | 0.7782    | 0.87      | 0.9786    | 0.7992    | 0.4615    | 0.7805    | 0.1183    | 0.6399    | 0.1434    | 0.9447    | 0.5218    | 0.4147    | 0.2646    | 0.7742    | 0.4562    | 0.5684    | 0.01879   | 0.6176    | 0.6121    | 0.6169    | 0.9437    | 0.6818    | 0.3595    | 0.437     | 0.6976    | 0.06023   | 0.6668    | 0.6706    | 0.2104    | 0.1289    | 0.3154    | 0.3637    | 0.5702    | 0.4386    | 0.9884    | 0.102     | 0.2089    | 0.1613    | 0.6531    | 0.2533    | 0.4663    | 0.2444    | 0.159     | 0.1104    | 0.6563    | 0.1382    | 0.1966    | 0.3687    | 0.821     | 0.0971    | 0.8379    | 0.0961    | 0.9765    | 0.4687    | 0.9768    | 0.6048    | 0.7393    | 0.03919   | 0.2828    | 0.1202    | 0.2961    | 0.1187    | 0.318     | 0.

Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:41<00:00, 24.54it/s]


...

Building TEST_KERNEL: 100%|██████████| 2500/2500 [03:04<00:00, 13.54it/s]


| 16        | 0.4665    | 0.9144    | 0.6412    | 0.163     | 0.3129    | 0.7809    | 0.9622    | 0.2629    | 0.165     | 0.2854    | 0.1682    | 0.7598    | 0.3027    | 0.9638    | 0.1357    | 0.82      | 0.5347    | 0.1966    | 0.2745    | 0.9572    | 0.9825    | 0.8705    | 0.9239    | 0.2857    | 0.3881    | 0.527     | 0.8594    | 0.02119   | 0.09553   | 0.2495    | 0.003422  | 0.3445    | 0.824     | 0.8854    | 0.7395    | 0.1561    | 0.6511    | 0.09021   | 0.1532    | 0.4193    | 0.3266    | 0.2786    | 0.1795    | 0.2142    | 0.9078    | 0.3146    | 0.1562    | 0.1326    | 0.09067   | 0.5784    | 0.7761    | 0.4496    | 0.2376    | 0.6117    | 0.996     | 0.2933    | 0.6977    | 0.328     | 0.4396    | 0.5474    | 0.238     | 0.6677    | 0.1852    | 0.01571   | 0.4997    | 0.8718    | 0.2379    | 0.6737    | 0.3552    | 0.5443    | 0.6218    | 0.6752    | 0.719     | 0.2158    | 0.1065    | 0.5813    | 0.5002    | 0.7255    | 0.6854    | 0.3366    | 0.7525    | 0.9883    | 0.

Building TEST_KERNEL: 100%|██████████| 2500/2500 [01:11<00:00, 34.93it/s]


| 17        | 0.4609    | 0.8896    | 0.03314   | 0.7435    | 0.3417    | 0.8171    | 0.06233   | 0.2381    | 0.4332    | 0.9591    | 0.499     | 0.1478    | 0.6019    | 0.5112    | 0.9169    | 0.1677    | 0.5142    | 0.3806    | 0.2881    | 0.488     | 0.5344    | 0.6507    | 0.9947    | 0.37      | 0.1148    | 0.7227    | 0.5329    | 0.2581    | 0.6835    | 0.1804    | 0.9166    | 0.9266    | 0.7724    | 0.7572    | 0.2206    | 0.4623    | 0.3366    | 0.2832    | 0.3201    | 0.1567    | 0.6963    | 0.1498    | 0.7347    | 0.2057    | 0.8446    | 0.6227    | 0.9686    | 0.5184    | 0.09903   | 0.9008    | 0.451     | 0.3593    | 0.5918    | 0.9333    | 0.7701    | 0.8065    | 0.7904    | 0.09302   | 0.4633    | 0.9423    | 0.9332    | 0.2063    | 0.5461    | 0.6532    | 0.4343    | 0.441     | 0.3511    | 0.1588    | 0.2451    | 0.7444    | 0.7277    | 0.09892   | 0.7411    | 0.266     | 0.3722    | 0.8221    | 0.8657    | 0.7058    | 0.195     | 0.7215    | 0.7733    | 0.8576    | 0.

Building TEST_KERNEL:  10%|█         | 258/2500 [00:10<01:40, 22.25it/s]